# 🎬 Supernan Hindi Dubbing Pipeline
**Flow:** Video → Whisper (Kannada→English) → Helsinki-NLP (English→Hindi) → gTTS → Wav2Lip

**Before running:**
1. Set Runtime → T4 GPU
2. Upload `dub_video.py` and your source video to `Drive/MyDrive/Supernan/`
3. Run all cells top to bottom

## Cell 1 — Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Move into your Supernan folder
%cd /content/drive/MyDrive/Supernan
!ls

## Cell 2 — Install Python 3.10

In [ ]:
%%bash
apt-get update -qq
apt-get install -y python3.10 python3.10-distutils python3.10-venv -qq
curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10
python3.10 --version

## Cell 3 — Install Python Dependencies
> ⚠️ **You only need to run this once per session.** If you restart the runtime, run it again.

In [ ]:
# Core ML/audio deps — pinned for Wav2Lip + transformers compatibility
!python3.10 -m pip install -q "numpy<2" "librosa==0.9.2" "transformers==4.44.2" sentencepiece

# TTS + translation
!python3.10 -m pip install -q gtts deep-translator

# Audio/video utils
!python3.10 -m pip install -q ffmpeg-python pydub

# PyTorch (Colab usually has this, but install just in case)
!python3.10 -m pip install -q torch torchvision torchaudio

# Wav2Lip deps
!python3.10 -m pip install -q opencv-python "scipy<1.13" "pillow<10" urllib3==1.26.6

# Whisper (OpenAI fork)
!python3.10 -m pip install -q git+https://github.com/openai/whisper.git

print('\n✅ All dependencies installed!')

## Cell 4 — Set Up Wav2Lip
> ⚠️ **Only run this once.** Skip if `Wav2Lip/` folder already exists in your Supernan Drive folder.

In [ ]:
import os

if not os.path.exists('Wav2Lip'):
    print('Cloning Wav2Lip...')
    !git clone https://github.com/Rudrabha/Wav2Lip.git
else:
    print('Wav2Lip folder already exists, skipping clone.')

# Face detection model
os.makedirs('Wav2Lip/face_detection/detection/sfd', exist_ok=True)
if not os.path.exists('Wav2Lip/face_detection/detection/sfd/s3fd.pth'):
    print('Downloading face detection model...')
    !wget -q "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" \
         -O "Wav2Lip/face_detection/detection/sfd/s3fd.pth"

# Wav2Lip GAN checkpoint
os.makedirs('Wav2Lip/checkpoints', exist_ok=True)
if not os.path.exists('Wav2Lip/checkpoints/wav2lip_gan.pth'):
    print('Downloading Wav2Lip GAN checkpoint (~700MB)...')
    !wget -q "https://huggingface.co/camenduru/Wav2Lip/resolve/main/checkpoints/wav2lip_gan.pth" \
         -O "Wav2Lip/checkpoints/wav2lip_gan.pth"

# ⬇️ Patch inference.py: handle missing face gracefully (reuse last detected box)
inf_path = 'Wav2Lip/inference.py'
if os.path.exists(inf_path):
    with open(inf_path, 'r') as f:
        content = f.read()

    old_snippet = """\tresults = []
\tpady1, pady2, padx1, padx2 = args.pads
\tfor rect, image in zip(predictions, images):
\t\tif rect is None:
\t\t\tcv2.imwrite('temp/faulty_frame.jpg', image) # check this frame where the face was not detected.
\t\t\traise ValueError('Face not detected! Ensure the video contains a face in all the frames.')"""

    new_snippet = """\tresults = []
\tpady1, pady2, padx1, padx2 = args.pads
\tlast_rect = None
\tfor rect, image in zip(predictions, images):
\t\tif rect is None:
\t\t\tif last_rect is not None:
\t\t\t\tprint("Warning: no face in frame, reusing last box.")
\t\t\t\trect = last_rect
\t\t\telse:
\t\t\t\tcv2.imwrite('temp/faulty_frame.jpg', image)
\t\t\t\traise ValueError('Face not detected in the very first frame! Ensure the video starts with a visible face.')
\t\tlast_rect = rect"""

    if old_snippet in content:
        content = content.replace(old_snippet, new_snippet)
        with open(inf_path, 'w') as f:
            f.write(content)
        print('✅ inference.py patched (face detection fix applied)')
    elif 'last_rect' in content:
        print('✅ inference.py already patched')
    else:
        print('⚠️  Could not auto-patch inference.py. Please upload the patched version manually.')

print('✅ Wav2Lip setup complete!')

## Cell 5 — Run the Dubbing Pipeline
Edit `START` and `END` to choose which 15-second clip you want to dub.

**Memory Tips:** If you get an OOM (Out of Memory) error:
1. Reduce `WAV2LIP_BATCH_SIZE` (e.g., 64, 32).
2. Increase `RESIZE_FACTOR` to 2 or 3 (lowers resolution but saves VRAM).

In [ ]:
### ── CONFIGURE HERE ──────────────────────────────────────────────
VIDEO_FILE  = "Hygiene - Kannada.mp4"   # your source video filename
WAV2LIP_ROOT = "/content/drive/MyDrive/Supernan/Wav2Lip"
START = 3    # seconds
END   = 18   # seconds  (must be START + ≥ 15)
RESIZE_FACTOR = 1       # 1=original, 2=half-res (faster, less VRAM)
WAV2LIP_BATCH_SIZE = 64 # lower if OOM (e.g. 64, 32)
### ────────────────────────────────────────────────────────────────

!rm -rf workdir  # clear any previous run

!python3.10 dub_video.py \
    --input_video  "{VIDEO_FILE}" \
    --work_dir     workdir \
    --start        {START} \
    --end          {END} \
    --wav2lip_root "{WAV2LIP_ROOT}" \
    --resize_factor {RESIZE_FACTOR} \
    --wav2lip_batch_size {WAV2LIP_BATCH_SIZE}

## Cell 5.1 — Troubleshooting OOM (Out of Memory)
If Cell 5 fails with `Exit -9`, it means the GPU ran out of memory. Try the configuration below instead:

In [ ]:
# Emergency low-memory run
!rm -rf workdir
!python3.10 dub_video.py \
    --input_video  "{VIDEO_FILE}" \
    --work_dir     workdir \
    --start        {START} \
    --end          {END} \
    --wav2lip_root "{WAV2LIP_ROOT}" \
    --resize_factor 2 \
    --wav2lip_batch_size 16 \
    --face_det_batch_size 4

## Cell 6 — Preview & Download Output

In [ ]:
from IPython.display import Video, display
import os

output = 'workdir/dubbed_output.mp4'
if os.path.exists(output):
    print(f'✅ Output size: {os.path.getsize(output)/1024:.0f} KB')
    display(Video(output, embed=True, width=640))
else:
    print('❌ Output file not found. Check the error output above.')

In [ ]:
# Download the output directly to your computer
from google.colab import files
files.download('workdir/dubbed_output.mp4')